In [2]:
import os
import xml.etree.ElementTree as ET
from PIL import Image

def get_image_size(image_path):
    with Image.open(image_path) as img:
        return img.size

def convert_yolo_to_retina(txt_file_path, output_folder, class_mapping):
    with open(txt_file_path, 'r') as file:
        yolo_labels = file.readlines()
    # Extract image_id from the txt file name
    image_id = os.path.splitext(os.path.basename(txt_file_path))[0]
    image_path = f"D:\\SagharGhaffari\\Insects\\images\\train\\{image_id}.JPG"

    # Get image size
    image_width, image_height = get_image_size(image_path)

    # Create XML structure
    root = ET.Element("annotation")
    folder = ET.SubElement(root, "folder")
    folder.text = "train"

    filename = ET.SubElement(root, "filename")
    filename.text = f"{image_id}.JPG"

    path = ET.SubElement(root, "path")
    path.text = image_path

    source = ET.SubElement(root, "source")
    database = ET.SubElement(source, "database")
    database.text = "Unknown"

    size = ET.SubElement(root, "size")
    width = ET.SubElement(size, "width")
    height = ET.SubElement(size, "height")
    depth = ET.SubElement(size, "depth")

    width.text = str(image_width)
    height.text = str(image_height)
    depth.text = "3"

    segmented = ET.SubElement(root, "segmented")
    segmented.text = "0"

    for label in yolo_labels:
        cls_id, x_center, y_center, width_y, height_y = map(float, label.split())
        x_min, y_min, x_max, y_max = (
            int((x_center - width_y / 2) * image_width),
            int((y_center - height_y / 2) * image_height),
            int((x_center + width_y / 2) * image_width),
            int((y_center + height_y / 2) * image_height),
        )

        obj = ET.SubElement(root, "object")
        name = ET.SubElement(obj, "name")
        pose = ET.SubElement(obj, "pose")
        truncated = ET.SubElement(obj, "truncated")
        difficult = ET.SubElement(obj, "difficult")
        bndbox = ET.SubElement(obj, "bndbox")
        xmin_elem = ET.SubElement(bndbox, "xmin")
        ymin_elem = ET.SubElement(bndbox, "ymin")
        xmax_elem = ET.SubElement(bndbox, "xmax")
        ymax_elem = ET.SubElement(bndbox, "ymax")

        name.text = class_mapping.get(str(int(cls_id)), "Unknown")
        pose.text = "Unspecified"
        truncated.text = "0"
        difficult.text = "0"
        xmin_elem.text = str(x_min)
        ymin_elem.text = str(y_min)
        xmax_elem.text = str(x_max)
        ymax_elem.text = str(y_max)

    # Write the XML file to the specified output folder
    output_path = os.path.join(output_folder, f"{image_id}.xml")
    tree = ET.ElementTree(root)
    tree.write(output_path)

    # Print the generated XML (optional)
    print(ET.tostring(root, encoding="unicode"))

def batch_convert_yolo_to_retina(input_folder, output_folder, class_mapping):
    # Iterate through all .txt files in the input folder
    for file_name in os.listdir(input_folder):
        if file_name.endswith(".txt"):
            txt_file_path = os.path.join(input_folder, file_name)
            convert_yolo_to_retina(txt_file_path, output_folder, class_mapping)

# Example usage
input_folder = "D:\\SagharGhaffari\\insects\\labels\\train"
output_folder = "D:\\SagharGhaffari\\insects\\RetinaNet\\try"
class_mapping = {
    '0': 'Chelicerata',
    '1': 'Coleoptera',
    '2': 'Diptera',
    '3': 'Hemiptera',
    '4': 'Hymenoptera',
    '5': 'Lepidoptera'
}

batch_convert_yolo_to_retina(input_folder, output_folder, class_mapping)


<annotation><folder>train</folder><filename>RIMG0003_1_Knautia arvensis agg.JPG</filename><path>D:\SagharGhaffari\Insects\images\train\RIMG0003_1_Knautia arvensis agg.JPG</path><source><database>Unknown</database></source><size><width>4608</width><height>3456</height><depth>3</depth></size><segmented>0</segmented></annotation>
<annotation><folder>train</folder><filename>RIMG0011_1_Knautia arvensis agg.JPG</filename><path>D:\SagharGhaffari\Insects\images\train\RIMG0011_1_Knautia arvensis agg.JPG</path><source><database>Unknown</database></source><size><width>4608</width><height>3456</height><depth>3</depth></size><segmented>0</segmented><object><name>Lepidoptera</name><pose>Unspecified</pose><truncated>0</truncated><difficult>0</difficult><bndbox><xmin>1013</xmin><ymin>1229</ymin><xmax>1817</xmax><ymax>1701</ymax></bndbox></object><object><name>Lepidoptera</name><pose>Unspecified</pose><truncated>0</truncated><difficult>0</difficult><bndbox><xmin>1027</xmin><ymin>1224</ymin><xmax>1829</

In [4]:
import os
import csv
import xml.etree.ElementTree as ET

def extract_xml_data(xml_file_path):
    tree = ET.parse(xml_file_path)
    root = tree.getroot()

    image_path = root.findtext(".//path")
    class_name = root.findtext(".//object/name")

    xmin = int(root.findtext(".//object/bndbox/xmin"))
    ymin = int(root.findtext(".//object/bndbox/ymin"))
    xmax = int(root.findtext(".//object/bndbox/xmax"))
    ymax = int(root.findtext(".//object/bndbox/ymax"))

    return image_path, xmin, ymin, xmax, ymax, class_name

def convert_xml_folder_to_csv(xml_folder, output_csv_path):
    with open(output_csv_path, 'w', newline='') as csv_file:
        csv_writer = csv.writer(csv_file)
        csv_writer.writerow(['Image Path', 'X1', 'Y1', 'X2', 'Y2', 'Class Name'])

        for xml_file_name in os.listdir(xml_folder):
            if xml_file_name.endswith(".xml"):
                xml_file_path = os.path.join(xml_folder, xml_file_name)
                image_path, xmin, ymin, xmax, ymax, class_name = extract_xml_data(xml_file_path)
                csv_writer.writerow([image_path, xmin, ymin, xmax, ymax, class_name])

# Example usage
xml_folder = "D:\\SagharGhaffari\\insects\\RetinaNet\\try"
output_csv_path = "D:\\SagharGhaffari\\insects\\RetinaNet\\try.csv"

convert_xml_folder_to_csv(xml_folder, output_csv_path)


TypeError: int() argument must be a string, a bytes-like object or a real number, not 'NoneType'

In [5]:
import os
import csv
import xml.etree.ElementTree as ET

def extract_xml_data(xml_file_path):
    tree = ET.parse(xml_file_path)
    root = tree.getroot()

    image_path = root.findtext(".//path")
    class_name = root.findtext(".//object/name")

    bndbox = root.find(".//object/bndbox")
    if bndbox is not None:
        xmin = int(bndbox.findtext("xmin") or 0)
        ymin = int(bndbox.findtext("ymin") or 0)
        xmax = int(bndbox.findtext("xmax") or 0)
        ymax = int(bndbox.findtext("ymax") or 0)
    else:
        xmin, ymin, xmax, ymax = 0, 0, 0, 0

    return image_path, xmin, ymin, xmax, ymax, class_name

def convert_xml_folder_to_csv(xml_folder, output_csv_path):
    with open(output_csv_path, 'w', newline='') as csv_file:
        csv_writer = csv.writer(csv_file)
        csv_writer.writerow(['image_path', 'xmin', 'ymin', 'xmax', 'ymax', 'class_name'])

        for xml_file_name in os.listdir(xml_folder):
            if xml_file_name.endswith(".xml"):
                xml_file_path = os.path.join(xml_folder, xml_file_name)
                image_path, xmin, ymin, xmax, ymax, class_name = extract_xml_data(xml_file_path)
                csv_writer.writerow([image_path, xmin, ymin, xmax, ymax, class_name])

# Example usage
xml_folder = "D:\\SagharGhaffari\\insects\\RetinaNet\\try"
output_csv_path = "D:\\SagharGhaffari\\insects\\RetinaNet\\try.csv"
convert_xml_folder_to_csv(xml_folder, output_csv_path)


In [6]:
import os
import xml.etree.ElementTree as ET
from PIL import Image

def get_image_size(image_path):
    with Image.open(image_path) as img:
        return img.size

def convert_yolo_to_retina(txt_file_path, output_folder, class_mapping):
    with open(txt_file_path, 'r') as file:
        yolo_labels = file.readlines()
    # Extract image_id from the txt file name
    image_id = os.path.splitext(os.path.basename(txt_file_path))[0]
    image_path = f"D:\\SagharGhaffari\\Insects\\images\\train\\{image_id}.JPG"

    # Get image size
    image_width, image_height = get_image_size(image_path)

    # Create XML structure
    root = ET.Element("annotation")
    folder = ET.SubElement(root, "folder")
    folder.text = "val"

    filename = ET.SubElement(root, "filename")
    filename.text = f"{image_id}.JPG"

    path = ET.SubElement(root, "path")
    path.text = image_path

    source = ET.SubElement(root, "source")
    database = ET.SubElement(source, "database")
    database.text = "Unknown"

    size = ET.SubElement(root, "size")
    width = ET.SubElement(size, "width")
    height = ET.SubElement(size, "height")
    depth = ET.SubElement(size, "depth")

    width.text = str(image_width)
    height.text = str(image_height)
    depth.text = "3"

    segmented = ET.SubElement(root, "segmented")
    segmented.text = "0"

    for label in yolo_labels:
        cls_id, x_center, y_center, width_y, height_y = map(float, label.split())
        x_min, y_min, x_max, y_max = (
            int((x_center - width_y / 2) * image_width),
            int((y_center - height_y / 2) * image_height),
            int((x_center + width_y / 2) * image_width),
            int((y_center + height_y / 2) * image_height),
        )

        obj = ET.SubElement(root, "object")
        name = ET.SubElement(obj, "name")
        pose = ET.SubElement(obj, "pose")
        truncated = ET.SubElement(obj, "truncated")
        difficult = ET.SubElement(obj, "difficult")
        bndbox = ET.SubElement(obj, "bndbox")
        xmin_elem = ET.SubElement(bndbox, "xmin")
        ymin_elem = ET.SubElement(bndbox, "ymin")
        xmax_elem = ET.SubElement(bndbox, "xmax")
        ymax_elem = ET.SubElement(bndbox, "ymax")

        name.text = class_mapping.get(str(int(cls_id)), "Unknown")
        pose.text = "Unspecified"
        truncated.text = "0"
        difficult.text = "0"
        xmin_elem.text = str(x_min)
        ymin_elem.text = str(y_min)
        xmax_elem.text = str(x_max)
        ymax_elem.text = str(y_max)

    # Write the XML file to the specified output folder
    output_path = os.path.join(output_folder, f"{image_id}.xml")
    tree = ET.ElementTree(root)
    tree.write(output_path)

    # Print the generated XML (optional)
    print(ET.tostring(root, encoding="unicode"))

def batch_convert_yolo_to_retina(input_folder, output_folder, class_mapping):
    # Create the output folder if it doesn't exist
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    # Iterate through all .txt files in the input folder
    for file_name in os.listdir(input_folder):
        if file_name.endswith(".txt"):
            txt_file_path = os.path.join(input_folder, file_name)
            convert_yolo_to_retina(txt_file_path, output_folder, class_mapping)

# Example usage
input_folder = "D:\\SagharGhaffari\\insects\\labels\\train"
output_folder = "D:\\SagharGhaffari\\insects\\retina_annotations\\train"
class_mapping = {
    '0': 'Chelicerata',
    '1': 'Coleoptera',
    '2': 'Diptera',
    '3': 'Hemiptera',
    '4': 'Hymenoptera',
    '5': 'Lepidoptera'
}

batch_convert_yolo_to_retina(input_folder, output_folder, class_mapping)


<annotation><folder>val</folder><filename>RIMG0003_1_Knautia arvensis agg.JPG</filename><path>D:\SagharGhaffari\Insects\images\train\RIMG0003_1_Knautia arvensis agg.JPG</path><source><database>Unknown</database></source><size><width>4608</width><height>3456</height><depth>3</depth></size><segmented>0</segmented></annotation>
<annotation><folder>val</folder><filename>RIMG0011_1_Knautia arvensis agg.JPG</filename><path>D:\SagharGhaffari\Insects\images\train\RIMG0011_1_Knautia arvensis agg.JPG</path><source><database>Unknown</database></source><size><width>4608</width><height>3456</height><depth>3</depth></size><segmented>0</segmented><object><name>Lepidoptera</name><pose>Unspecified</pose><truncated>0</truncated><difficult>0</difficult><bndbox><xmin>1013</xmin><ymin>1229</ymin><xmax>1817</xmax><ymax>1701</ymax></bndbox></object><object><name>Lepidoptera</name><pose>Unspecified</pose><truncated>0</truncated><difficult>0</difficult><bndbox><xmin>1027</xmin><ymin>1224</ymin><xmax>1829</xmax

In [9]:
import os
import csv
import xml.etree.ElementTree as ET

def extract_xml_data(xml_file_path):
    tree = ET.parse(xml_file_path)
    root = tree.getroot()

    image_path = root.findtext(".//path")
    class_name = root.findtext(".//object/name")

    xmin = int(root.findtext(".//object/bndbox/xmin"))
    ymin = int(root.findtext(".//object/bndbox/ymin"))
    xmax = int(root.findtext(".//object/bndbox/xmax"))
    ymax = int(root.findtext(".//object/bndbox/ymax"))

    return image_path, xmin, ymin, xmax, ymax, class_name

def convert_xml_folder_to_csv(xml_folder, output_csv_path):
    with open(output_csv_path, 'w', newline='') as csv_file:
        csv_writer = csv.writer(csv_file)
        csv_writer.writerow(['Image Path', 'X1', 'Y1', 'X2', 'Y2', 'Class Name'])

        for xml_file_name in os.listdir(xml_folder):
            if xml_file_name.endswith(".xml"):
                xml_file_path = os.path.join(xml_folder, xml_file_name)
                image_path, xmin, ymin, xmax, ymax, class_name = extract_xml_data(xml_file_path)
                csv_writer.writerow([image_path, xmin, ymin, xmax, ymax, class_name])

# Example usage
xml_folder = "D:\SagharGhaffari\insects\RetinaNet\\retina_annotations\\train"
output_csv_path = "D:\\SagharGhaffari\\insects\RetinaNet\\train_annotations.csv"

convert_xml_folder_to_csv(xml_folder, output_csv_path)


In [10]:
import os
import xml.etree.ElementTree as ET
from PIL import Image

def get_image_size(image_path):
    with Image.open(image_path) as img:
        return img.size

def convert_yolo_to_retina(txt_file_path, output_folder, class_mapping):
    with open(txt_file_path, 'r') as file:
        yolo_labels = file.readlines()
    # Extract image_id from the txt file name
    image_id = os.path.splitext(os.path.basename(txt_file_path))[0]
    image_path = f"D:\\SagharGhaffari\\Insects\\images\\train\\{image_id}.JPG"

    # Get image size
    image_width, image_height = get_image_size(image_path)

    # Create XML structure
    root = ET.Element("annotation")
    folder = ET.SubElement(root, "folder")
    folder.text = "train"

    filename = ET.SubElement(root, "filename")
    filename.text = f"{image_id}.JPG"

    path = ET.SubElement(root, "path")
    path.text = image_path

    source = ET.SubElement(root, "source")
    database = ET.SubElement(source, "database")
    database.text = "Unknown"

    size = ET.SubElement(root, "size")
    width = ET.SubElement(size, "width")
    height = ET.SubElement(size, "height")
    depth = ET.SubElement(size, "depth")

    width.text = str(image_width)
    height.text = str(image_height)
    depth.text = "3"

    segmented = ET.SubElement(root, "segmented")
    segmented.text = "0"

    for label in yolo_labels:
        cls_id, x_center, y_center, width_y, height_y = map(float, label.split())
        x_min, y_min, x_max, y_max = (
            int((x_center - width_y / 2) * image_width),
            int((y_center - height_y / 2) * image_height),
            int((x_center + width_y / 2) * image_width),
            int((y_center + height_y / 2) * image_height),
        )

        obj = ET.SubElement(root, "object")
        name = ET.SubElement(obj, "name")
        pose = ET.SubElement(obj, "pose")
        truncated = ET.SubElement(obj, "truncated")
        difficult = ET.SubElement(obj, "difficult")
        bndbox = ET.SubElement(obj, "bndbox")
        xmin_elem = ET.SubElement(bndbox, "xmin")
        ymin_elem = ET.SubElement(bndbox, "ymin")
        xmax_elem = ET.SubElement(bndbox, "xmax")
        ymax_elem = ET.SubElement(bndbox, "ymax")

        name.text = class_mapping.get(str(int(cls_id)), "Unknown")
        pose.text = "Unspecified"
        truncated.text = "0"
        difficult.text = "0"
        xmin_elem.text = str(x_min)
        ymin_elem.text = str(y_min)
        xmax_elem.text = str(x_max)
        ymax_elem.text = str(y_max)

    # Write the XML file to the specified output folder
    output_path = os.path.join(output_folder, f"{image_id}.xml")
    tree = ET.ElementTree(root)
    tree.write(output_path)

    # Print the generated XML (optional)
    print(ET.tostring(root, encoding="unicode"))

def batch_convert_yolo_to_retina(input_folder, output_folder, class_mapping):
    # Iterate through all .txt files in the input folder
    for file_name in os.listdir(input_folder):
        if file_name.endswith(".txt"):
            txt_file_path = os.path.join(input_folder, file_name)
            convert_yolo_to_retina(txt_file_path, output_folder, class_mapping)

# Example usage
input_folder = "D:\SagharGhaffari\insects\images\Retina\JPEGImages"
output_folder = "D:\\SagharGhaffari\\insects\\retina_annotations\\train"
class_mapping = {
    '0': 'Chelicerata',
    '1': 'Coleoptera',
    '2': 'Diptera',
    '3': 'Hemiptera',
    '4': 'Hymenoptera',
    '5': 'Lepidoptera'
}

batch_convert_yolo_to_retina(input_folder, output_folder, class_mapping)
